In [1]:
%matplotlib inline

In [ ]:
style_img_path = "./data/9styles/composition_vii.jpg"
content_img_path = "./data/images/neural-style/Ben Reilly - Scarlet Spider (2017-) 016-010.jpg"
style_transfer_output_path = "woman-with-hat-matisse_vgg.png"

# test_data_path = '/home/test/Documents/express_recognition/data/icdar2015/test2015'
test_data_path = './testdata'
checkpoint_path = './checkpoints_total/model_220.pth'
output_dir_box = './result/box'
output_dir_txt = './result/txt'
output_dir_pic = './result/pic'


Neural Transfer Using PyTorch
=============================


**Author**: `Alexis Jacq <https://alexis-jacq.github.io>`_
 
**Edited by**: `Winston Herring <https://github.com/winston6>`_

Introduction
------------

This tutorial explains how to implement the `Neural-Style algorithm <https://arxiv.org/abs/1508.06576>`__
developed by Leon A. Gatys, Alexander S. Ecker and Matthias Bethge.
Neural-Style, or Neural-Transfer, allows you to take an image and
reproduce it with a new artistic style. The algorithm takes three images,
an input image, a content-image, and a style-image, and changes the input 
to resemble the content of the content-image and the artistic style of the style-image.

 
.. figure:: /_static/img/neural-style/neuralstyle.png
   :alt: content1



Underlying Principle
--------------------

The principle is simple: we define two distances, one for the content
($D_C$) and one for the style ($D_S$). $D_C$ measures how different the content
is between two images while $D_S$ measures how different the style is
between two images. Then, we take a third image, the input, and
transform it to minimize both its content-distance with the
content-image and its style-distance with the style-image. Now we can
import the necessary packages and begin the neural transfer.

Importing Packages and Selecting a Device
-----------------------------------------
Below is a  list of the packages needed to implement the neural transfer.

-  ``torch``, ``torch.nn``, ``numpy`` (indispensables packages for
   neural networks with PyTorch)
-  ``torch.optim`` (efficient gradient descents)
-  ``PIL``, ``PIL.Image``, ``matplotlib.pyplot`` (load and display
   images)
-  ``torchvision.transforms`` (transform PIL images into tensors)
-  ``torchvision.models`` (train or load pre-trained models)
-  ``copy`` (to deep copy the models; system package)



In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import copy

Next, we need to choose which device to run the network on and import the
content and style images. Running the neural transfer algorithm on large
images takes longer and will go much faster when running on a GPU. We can
use ``torch.cuda.is_available()`` to detect if there is a GPU available.
Next, we set the ``torch.device`` for use throughout the tutorial. Also the ``.to(device)``
method is used to move tensors or modules to a desired device. 



In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading the Images
------------------

Now we will import the style and content images. The original PIL images have values between 0 and 255, but when
transformed into torch tensors, their values are converted to be between
0 and 1. The images also need to be resized to have the same dimensions.
An important detail to note is that neural networks from the
torch library are trained with tensor values ranging from 0 to 1. If you
try to feed the networks with 0 to 255 tensor images, then the activated
feature maps will be unable sense the intended content and style.
However, pre-trained networks from the Caffe library are trained with 0
to 255 tensor images. 


.. Note::
    Here are links to download the images required to run the tutorial:
    `picasso.jpg <https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg>`__ and
    `dancing.jpg <https://pytorch.org/tutorials/_static/img/neural-style/dancing.jpg>`__.
    Download these two images and add them to a directory
    with name ``images`` in your current working directory.



In [4]:
# desired size of the output image
imsize = 1024 if torch.cuda.is_available() else 128  # use small size if no gpu

loader = transforms.Compose([
    transforms.Resize(imsize),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


style_img = image_loader(style_img_path)
content_img = image_loader()
print(style_img.size())
print(content_img.size())

assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"

torch.Size([1, 3, 196, 128])
torch.Size([1, 3, 196, 128])


Now, let's create a function that displays an image by reconverting a 
copy of it to PIL format and displaying the copy using 
``plt.imshow``. We will try displaying the content and style images 
to ensure they were imported correctly.



In [5]:
unloader = transforms.ToPILImage()  # reconvert into PIL image

plt.ion()

def imshow(tensor, title=None):
    image = tensor.cpu().clone()  # we clone the tensor to not do changes on it
    image = image.squeeze(0)      # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated


plt.figure()
imshow(style_img, title='Style Image')

plt.figure()
imshow(content_img, title='Content Image')

Loss Functions
--------------
Content Loss
~~~~~~~~~~~~

The content loss is a function that represents a weighted version of the
content distance for an individual layer. The function takes the feature
maps $F_{XL}$ of a layer $L$ in a network processing input $X$ and returns the
weighted content distance $w_{CL}.D_C^L(X,C)$ between the image $X$ and the
content image $C$. The feature maps of the content image($F_{CL}$) must be
known by the function in order to calculate the content distance. We
implement this function as a torch module with a constructor that takes
$F_{CL}$ as an input. The distance $\|F_{XL} - F_{CL}\|^2$ is the mean square error
between the two sets of feature maps, and can be computed using ``nn.MSELoss``.

We will add this content loss module directly after the convolution
layer(s) that are being used to compute the content distance. This way
each time the network is fed an input image the content losses will be
computed at the desired layers and because of auto grad, all the
gradients will be computed. Now, in order to make the content loss layer
transparent we must define a ``forward`` method that computes the content
loss and then returns the layer’s input. The computed loss is saved as a
parameter of the module.




In [6]:
class ContentLoss(nn.Module):

    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        # we 'detach' the target content from the tree used
        # to dynamically compute the gradient: this is a stated value,
        # not a variable. Otherwise the forward method of the criterion
        # will throw an error.
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

.. Note::
   **Important detail**: although this module is named ``ContentLoss``, it
   is not a true PyTorch Loss function. If you want to define your content
   loss as a PyTorch Loss function, you have to create a PyTorch autograd function 
   to recompute/implement the gradient manually in the ``backward``
   method.



Style Loss
~~~~~~~~~~

The style loss module is implemented similarly to the content loss
module. It will act as a transparent layer in a
network that computes the style loss of that layer. In order to
calculate the style loss, we need to compute the gram matrix $G_{XL}$. A gram
matrix is the result of multiplying a given matrix by its transposed
matrix. In this application the given matrix is a reshaped version of
the feature maps $F_{XL}$ of a layer $L$. $F_{XL}$ is reshaped to form $\hat{F}_{XL}$, a $K$\ x\ $N$
matrix, where $K$ is the number of feature maps at layer $L$ and $N$ is the
length of any vectorized feature map $F_{XL}^k$. For example, the first line
of $\hat{F}_{XL}$ corresponds to the first vectorized feature map $F_{XL}^1$.

Finally, the gram matrix must be normalized by dividing each element by
the total number of elements in the matrix. This normalization is to
counteract the fact that $\hat{F}_{XL}$ matrices with a large $N$ dimension yield
larger values in the Gram matrix. These larger values will cause the
first layers (before pooling layers) to have a larger impact during the
gradient descent. Style features tend to be in the deeper layers of the
network so this normalization step is crucial.




In [7]:
def gram_matrix(input):
    a, b, c, d = input.size()  # a=batch size(=1)
    # b=number of feature maps
    # (c,d)=dimensions of a f. map (N=c*d)

    features = input.view(a * b, c * d)  # resise F_XL into \hat F_XL

    G = torch.mm(features, features.t())  # compute the gram product

    # we 'normalize' the values of the gram matrix
    # by dividing by the number of element in each feature maps.
    return G.div(a * b * c * d)

Now the style loss module looks almost exactly like the content loss
module. The style distance is also computed using the mean square
error between $G_{XL}$ and $G_{SL}$.




In [8]:
class StyleLoss(nn.Module):

    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

Importing the Model
-------------------

Now we need to import a pre-trained neural network. We will use a 19
layer VGG network like the one used in the paper.

PyTorch’s implementation of VGG is a module divided into two child
``Sequential`` modules: ``features`` (containing convolution and pooling layers),
and ``classifier`` (containing fully connected layers). We will use the
``features`` module because we need the output of the individual
convolution layers to measure content and style loss. Some layers have
different behavior during training than evaluation, so we must set the
network to evaluation mode using ``.eval()``.




In [9]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

Additionally, VGG networks are trained on images with each channel
normalized by mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
We will use them to normalize the image before sending it into the network.




In [10]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

A ``Sequential`` module contains an ordered list of child modules. For
instance, ``vgg19.features`` contains a sequence (Conv2d, ReLU, MaxPool2d,
Conv2d, ReLU…) aligned in the right order of depth. We need to add our
content loss and style loss layers immediately after the convolution
layer they are detecting. To do this we must create a new ``Sequential``
module that has content loss and style loss modules correctly inserted.




In [11]:
# desired depth layers to compute style/content losses :
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    cnn = copy.deepcopy(cnn)

    # normalization module
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    # just in order to have an iterable access to or list of content/syle
    # losses
    content_losses = []
    style_losses = []

    # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
    # to put in modules that are supposed to be activated sequentially
    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            # The in-place version doesn't play very nicely with the ContentLoss
            # and StyleLoss we insert below. So we replace with out-of-place
            # ones here.
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            # add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            # add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses

Next, we select the input image. You can use a copy of the content image
or white noise.




In [12]:
input_img = content_img.clone()
# if you want to use white noise instead uncomment the below line:
# input_img = torch.randn(content_img.data.size(), device=device)

# add the original input image to the figure:
plt.figure()
imshow(input_img, title='Input Image')

Gradient Descent
----------------

As Leon Gatys, the author of the algorithm, suggested `here <https://discuss.pytorch.org/t/pytorch-tutorial-for-neural-transfert-of-artistic-style/336/20?u=alexis-jacq>`__, we will use
L-BFGS algorithm to run our gradient descent. Unlike training a network,
we want to train the input image in order to minimise the content/style
losses. We will create a PyTorch L-BFGS optimizer ``optim.LBFGS`` and pass
our image to it as the tensor to optimize.




In [13]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer

Finally, we must define a function that performs the neural transfer. For
each iteration of the networks, it is fed an updated input and computes
new losses. We will run the ``backward`` methods of each loss module to
dynamicaly compute their gradients. The optimizer requires a “closure”
function, which reevaluates the modul and returns the loss.

We still have one final constraint to address. The network may try to
optimize the input with values that exceed the 0 to 1 tensor range for
the image. We can address this by correcting the input values to be
between 0 to 1 each time the network is run.




In [14]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img, num_steps=300,
                       style_weight=1000000, content_weight=1):
    """Run the style transfer."""
    print('Building the style transfer model..')
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)
    optimizer = get_input_optimizer(input_img)

    print('Optimizing..')
    run = [0]
    while run[0] <= num_steps:

        def closure():
            # correct the values of updated input image
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            if run[0] % 50 == 0:
                print("run {}:".format(run))
                print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                    style_score.item(), content_score.item()))
                print()

            return style_score + content_score

        optimizer.step(closure)

    # a last correction...
    input_img.data.clamp_(0, 1)

    return input_img

Finally, we can run the algorithm.




Building the style transfer model..


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Optimizing..
run [50]:
Style Loss : 111.610298 Content Loss: 32.061642

run [100]:
Style Loss : 37.610214 Content Loss: 33.149406

run [150]:
Style Loss : 19.608473 Content Loss: 31.712284

run [200]:
Style Loss : 13.253098 Content Loss: 29.699394

run [250]:
Style Loss : 10.174708 Content Loss: 28.067982

run [300]:
Style Loss : 8.585974 Content Loss: 26.790230



In [15]:
output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img)

plt.figure()
imshow(output, title='Output Image')

# sphinx_gallery_thumbnail_number = 4
plt.ioff()
plt.show()

In [16]:
import torchvision.utils as utils
utils.save_image(output, style_transfer_output_path)

## text detection


In [17]:
import cv2
import time
import math
import os 
import numpy as np

import locality_aware_nms as nms_locality
# import lanms

import torch
import model
from data_utils import restore_rectangle
from torch.autograd import Variable


import model


def rotate(box_List,image):
	#xuan zhuan tu pian

	n=len(box_List)
	c=0;
	angle=0
	for i in range(n):
		box=box_List[i]
		y1 = min(box[0][1], box[1][1], box[2][1], box[3][1])
		y2 = max(box[0][1], box[1][1], box[2][1], box[3][1])
		x1 = min(box[0][0], box[1][0], box[2][0], box[3][0])
		x2 = max(box[0][0], box[1][0], box[2][0], box[3][0])
		for j in range(4):
			if(box[j][1]==y2):
				k1=j
		for j in range(4):
			if(box[j][0]==x2 and j!=k1):
				k2=j
		c=(box[k1][0]-box[k2][0])*1.0/(box[k1][1]-box[k2][1])
		if(c<0):
			c=-c
		if(c>1):
			c=1.0/c
		angle=math.atan(c)+angle
	angle=angle/n
	(h, w) = image.shape[:2]
	center = (w / 2, h / 2)
	scale=1
	M = cv2.getRotationMatrix2D(center,angle, scale)
	image_new = cv2.warpAffine(image, M, (w, h))
	return image_new

def get_images():
	'''
	find image files in test data path
	:return: list of files found
	'''
	files = []
	exts = ['jpg', 'png', 'jpeg', 'JPG']
	for parent, dirnames, filenames in os.walk(test_data_path):
		for filename in filenames:
			for ext in exts:
				if filename.endswith(ext):
					files.append(os.path.join(parent, filename))
					break
	# print('Find {} images'.format(len(files)))
	return files

def resize_image(im, max_side_len=2400):
	'''
	resize image to a size multiple of 32 which is required by the network
	:param im: the resized image
	:param max_side_len: limit of max image size to avoid out of memory in gpu
	:return: the resized image and the resize ratio
	'''
	h, w, _ = im.shape

	resize_w = w
	resize_h = h

	# limit the max side
	if max(resize_h, resize_w) > max_side_len:
		ratio = float(max_side_len) / resize_h if resize_h > resize_w else float(max_side_len) / resize_w
	else:
		ratio = 1.
	resize_h = int(resize_h * ratio)
	resize_w = int(resize_w * ratio)

	resize_h = resize_h if resize_h % 32 == 0 else (resize_h // 32 - 1) * 32
	resize_w = resize_w if resize_w % 32 == 0 else (resize_w // 32 - 1) * 32
	im = cv2.resize(im, (int(resize_w), int(resize_h)))

	ratio_h = resize_h / float(h)
	ratio_w = resize_w / float(w)

	return im, (ratio_h, ratio_w)

def detect(score_map, geo_map, timer, score_map_thresh=0.8, box_thresh=0.1, nms_thres=0.2):
	'''
	restore text boxes from score map and geo map
	:param score_map:
	:param geo_map:
	:param timer:
	:param score_map_thresh: threshhold for score map
	:param box_thresh: threshhold for boxes
	:param nms_thres: threshold for nms
	:return:
	'''
	
	if len(score_map.shape) == 4:
		score_map = score_map[0, :, :, 0]
		geo_map = geo_map[0, :, :, ]
	# filter the score map
	xy_text = np.argwhere(score_map > score_map_thresh)
	# sort the text boxes via the y axis
	xy_text = xy_text[np.argsort(xy_text[:, 0])]
	# restore
	start = time.time()
	text_box_restored = restore_rectangle(xy_text[:, ::-1]*4, geo_map[xy_text[:, 0], xy_text[:, 1], :]) # N*4*2
	# print('{} text boxes before nms'.format(text_box_restored.shape[0]))
	boxes = np.zeros((text_box_restored.shape[0], 9), dtype=np.float32)
	boxes[:, :8] = text_box_restored.reshape((-1, 8))
	boxes[:, 8] = score_map[xy_text[:, 0], xy_text[:, 1]]
	timer['restore'] = time.time() - start
	# nms part
	start = time.time()
	boxes = nms_locality.nms_locality(boxes.astype(np.float64), nms_thres)
# 	boxes = lanms.merge_quadrangle_n9(boxes.astype('float32'), nms_thres)
	timer['nms'] = time.time() - start

	if boxes.shape[0] == 0:
		return None, timer

	# here we filter some low score boxes by the average score map, this is different from the orginal paper
	for i, box in enumerate(boxes):
		mask = np.zeros_like(score_map, dtype=np.uint8)
		cv2.fillPoly(mask, box[:8].reshape((-1, 4, 2)).astype(np.int32) // 4, 1)
		boxes[i, 8] = cv2.mean(score_map, mask)[0]
	boxes = boxes[boxes[:, 8] > box_thresh]

	return boxes, timer

def sort_poly(p):
	min_axis = np.argmin(np.sum(p, axis=1))
	p = p[[min_axis, (min_axis+1)%4, (min_axis+2)%4, (min_axis+3)%4]]
	if abs(p[0, 0] - p[1, 0]) > abs(p[0, 1] - p[1, 1]):
		return p
	else:
		return p[[0, 3, 2, 1]]

def change_box(box_List):
	n=len(box_List)
	for i in range(n):
		box=box_List[i]
		y1 = min(box[0][1], box[1][1], box[2][1], box[3][1])
		y2 = max(box[0][1], box[1][1], box[2][1], box[3][1])
		x1 = min(box[0][0], box[1][0], box[2][0], box[3][0])
		x2 = max(box[0][0], box[1][0], box[2][0], box[3][0])
		box[0][1]=y1
		box[0][0]=x1
		box[1][1]=y1
		box[1][0]=x2
		box[3][1]=y2
		box[3][0]=x1
		box[2][1]=y2
		box[2][0]=x2
		box_List[i]=box
	return box_List

def save_box(box_List,image,img_path):
	n=len(box_List)
	box_final = []
	for i in range(n):
		box=box_List[i]
		y1_0 = int(min(box[0][1], box[1][1], box[2][1], box[3][1]))
		y2_0 = int(max(box[0][1], box[1][1], box[2][1], box[3][1]))
		x1_0 = int(min(box[0][0], box[1][0], box[2][0], box[3][0]))
		x2_0 = int(max(box[0][0], box[1][0], box[2][0], box[3][0]))
		y1 = max(int(y1_0 - 0.1 * (y2_0 - y1_0)), 0)
		y2 = min(int(y2_0 + 0.1 * (y2_0 - y1_0)), image.shape[0] - 1)
		x1 = max(int(x1_0 - 0.25 * (x2_0 - x1_0)), 0)
		x2 = min(int(x2_0 + 0.25 * (x2_0 - x1_0)), image.shape[1] - 1)
		image_new=image[y1:y2,x1:x2]

		# # 图像处理
		gray_2 = image_new[:,:,0]
		gradX = cv2.Sobel(gray_2, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
		gradY = cv2.Sobel(gray_2, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
		blurred = cv2.blur(gradX, (2, 2))
		(_, thresh) = cv2.threshold(blurred, 160, 255, cv2.THRESH_BINARY)
		# closed = cv2.erode(thresh, None, iterations = 1)
		# closed = cv2.dilate(closed, None, iterations = 1)
		closed = thresh
		x_plus = []
		x_left = 1
		x_right = closed.shape[1]
		for jj in range(0, closed.shape[1]):
			plus = 0
			for ii in range(0, closed.shape[0]):
				plus = plus + closed[ii][jj]
			x_plus.append(plus)
		
		for jj in range(0, int(closed.shape[1] * 0.5 - 1)):
			if(x_plus[jj] > 0.4 * max(x_plus)):
				x_left = max(jj - 5, 0)
				break
		for ii in range(closed.shape[1] - 1, int(closed.shape[1] * 0.5 + 1), -1):
			if(x_plus[ii] > 0.4 * max(x_plus)):
				x_right = min(ii + 5, closed.shape[1] - 1)
				break
		
		image_new = image_new[:, x_left:x_right]
		cv2.imwrite("." + img_path.split(".")[1]+'_'+str(i)+".jpg", image_new)
		box[0][1]=y1
		box[0][0]=x1 + x_left
		box[1][1]=y1
		box[1][0]=x1 + x_right
		box[3][1]=y2
		box[3][0]=x1 + x_left
		box[2][1]=y2
		box[2][0]=x1 + x_right
		box_List[i]=box
	return box_List

East_model = model.East()
East_model = East_model.eval()
East_model = East_model.cuda()

East_model.load_state_dict(torch.load(checkpoint_path))

def predict(argv=None):
	
    try:
        os.makedirs(output_dir_box)
        os.makedirs(output_dir_txt)
        os.makedirs(output_dir_pic)
    except OSError as e:
        if e.errno != 17:
            raise
    
    im_fn_list = get_images()
    start = time.time()
    for im_fn in im_fn_list:
        # print(im_fn)
        im = cv2.imread(im_fn)[:, :, ::-1]
        start_time = time.time()
        im_resized, (ratio_h, ratio_w) = resize_image(im)
        im_resized = im_resized.astype(np.float32)
        im_resized = Variable(torch.from_numpy(im_resized)).cuda()
        im_resized = im_resized.unsqueeze(0)
        im_resized = im_resized.permute(0, 3, 1, 2)
        
        timer = {'net': 0, 'restore': 0, 'nms': 0}
       
        score, geometry = East_model(im_resized)
        score = score.permute(0, 2, 3, 1)
        geometry = geometry.permute(0, 2, 3, 1)
        score = score.data.cpu().numpy()
        geometry = geometry.data.cpu().numpy()
        
        
        boxes, timer = detect(score_map=score, geo_map=geometry, timer=timer)
        

        if boxes is not None:
            boxes = boxes[:, :8].reshape((-1, 4, 2))
            boxes[:, :, 0] /= ratio_w
            boxes[:, :, 1] /= ratio_h
        
        
        if boxes is not None:
            res_file = os.path.join(output_dir_txt, '{}.txt'.format(
                                os.path.basename(im_fn).split('.')[0]))
            
            with open(res_file, 'w') as f:
                for box in boxes:
                    
                    box = sort_poly(box.astype(np.int32))
                    if np.linalg.norm(box[0] - box[1]) < 5 or np.linalg.norm(box[3] - box[0]) < 5:
                        continue
                    f.write('{}, {}, {}, {}, {}, {}, {}, {}\r\n'.format(
                        box[0, 0], box[0, 1], box[1, 0], box[1, 1], box[2, 0], box[2, 1], box[3, 0], box[3, 1]))
                    cv2.polylines(im[:, :, ::-1], [box.astype(np.int32).reshape((-1, 1, 2))], True,
                                  color=(0, 0, 255), thickness=5)
             
        img_path = os.path.join(output_dir_pic, os.path.basename(im_fn))
        cv2.imwrite(img_path, im[:, :, ::-1])

    during = time.time() - start
    print('average :{:.6f}'.format(during / len(im_fn_list)))

if __name__ == "__main__":
	predict()


ModuleNotFoundError: No module named 'locality_aware_nms'

In [ ]:
predict()

In [ ]:
## merge the two result

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def get_mask(textfilepath, imgsize):
	with open(textfilepath, 'r') as f:
		polys = f.read()[:-1].split('\n')
		polys = [poly.split(', ') for poly in polys]
		polys = [np.array(poly, dtype=np.int32).reshape((-1, 1, 2)) for poly in polys]

	mask = np.zeros(imgsize, dtype=np.uint8)

	if len(polys) == 0:
		return mask

	cv2.fillPoly(mask, polys, color=255)
	mask = mask.astype(bool)
	# cv2.polylines(mask, polys, True, color=255, thickness=1)

	return mask

if __name__ == '__main__':
	img = cv2.imread(content_img_path)
	m = get_mask('MCCALL_ROBINHOOD_T31_005.txt', img.shape[:2])
	print(m)
	m = np.repeat(m[:, :, np.newaxis], 3, axis=2)
	temp = np.zeros(img.shape, dtype=np.uint8)
	temp[np.where(m == 1)] = img[np.where(m == 1)]
	plt.figure(figsize = (16,12))
	plt.imshow(temp)